---
## Python Packages & Directories
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind")

# from smh import Session

import spag.read_data as rd
from spag.convert import *
from spag.utils import *
import spag.coordinates as coord

# import alexmods.read_data as rd


script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data/'


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [3]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Restructuring `Supplementary Data 1` Table to Merge with other Chiti_2024 Data
---


In [4]:
sup1 = pd.read_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2024/Supplementary_Data_1.csv")

rows = []
for name in sup1['name'].unique():
    row = {}
    if name == 'lmc3':
        row['Name'] = name.replace("lmc3", "LMC-003")
    else:
        row['Name'] = name.replace("lmc", "LMC-")
    row['Loc'] = 'DW'

    species_list = list(sup1[sup1['name'] == name]['Element'])
    symbols = list(sup1[sup1['name'] == name]['Element'].apply(species_to_element))
    
    elems = []
    for sym in symbols:
        if sym == 'C-H':
            elems += ['C']
        elif sym not in ['Ca II','Ti II','V II','Cr II','Mn II','Fe II']:
            elems += [sym.split(' ')[0]]
        else:
            elems += [''.join(sym.split(' '))]

    elements = {species_list[i]: elems[i] for i in range(len(species_list))}

    for species, elem in elements.items():
        subset = sup1[(sup1['name'] == name) & (sup1['Element'] == species)]
        if subset.empty:
            print("ERROR")
            continue
        
        row['eps{}'.format(elem[0].lower() + elem[1:])] = subset['Logeps'].values[0]
        if subset['ul'].values[0] == 0:
            row['[{}/H]'.format(elem)] = subset['[X/H]'].values[0]
            row['ul[{}/H]'.format(elem)] = np.nan
        elif subset['ul'].values[0] == 1:
            row['[{}/H]'.format(elem)] = np.nan
            row['ul[{}/H]'.format(elem)] = subset['[X/H]'].values[0]
        else:
            raise ValueError("'ul' value is not 0 or 1")
        # row['[{}/H]_err'.format(elem)] = subset['[X/H]_err'].values[0]
        # row['[{}/H]_errteff'.format(elem)] = subset['[X/H]_errteff'].values[0]
        # row['[{}/H]_errlogg'.format(elem)] = subset['[X/H]_errlogg'].values[0]
        # row['[{}/H]_errvt'.format(elem)] = subset['[X/H]_errvt'].values[0]
        # row['[{}/H]_errsys'.format(elem)] = subset['[X/H]_errsys'].values[0]
        # row['[{}/H]_errtot'.format(elem)] = subset['[X/H]_errtot'].values[0]
        row['e_[{}/H]'.format(elem)] = subset['[X/H]_errtot'].values[0]  # errtot
        row['[{}/Fe]'.format(elem)] = subset['[X/Fe]'].values[0]
        # row['[{}/Fe]_errteff'.format(elem)] = subset['[X/Fe]_errteff'].values[0]
        # row['[{}/Fe]_errlogg'.format(elem)] = subset['[X/Fe]_errlogg'].values[0]
        # row['[{}/Fe]_errvt'.format(elem)] = subset['[X/Fe]_errvt'].values[0]
        # row['[{}/Fe]_errsys'.format(elem)] = subset['[X/Fe]_errsys'].values[0]
        # row['[{}/Fe]_errtot'.format(elem)] = subset['[X/Fe]_errtot'].values[0]
        row['e_[{}/Fe]'.format(elem)] = subset['[X/Fe]_errtot'].values[0]  # errtot

    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2024/Supplementary_Data_1_yelland.csv", index=False)
display(df)



,Name,Loc,epsna,[Na/H],ul[Na/H],e_[Na/H],[Na/Fe],e_[Na/Fe],epsmg,[Mg/H],ul[Mg/H],e_[Mg/H],[Mg/Fe],e_[Mg/Fe],epsal,[Al/H],ul[Al/H],e_[Al/H],[Al/Fe],e_[Al/Fe],epssi,[Si/H],ul[Si/H],e_[Si/H],[Si/Fe],e_[Si/Fe],epsk,[K/H],ul[K/H],e_[K/H],[K/Fe],e_[K/Fe],epsca,[Ca/H],ul[Ca/H],e_[Ca/H],[Ca/Fe],e_[Ca/Fe],epssc,[Sc/H],ul[Sc/H],e_[Sc/H],[Sc/Fe],e_[Sc/Fe],epsti,[Ti/H],ul[Ti/H],e_[Ti/H],[Ti/Fe],e_[Ti/Fe],epstiII,[TiII/H],ul[TiII/H],e_[TiII/H],[TiII/Fe],e_[TiII/Fe],epsv,[V/H],ul[V/H],e_[V/H],[V/Fe],e_[V/Fe],epsvII,[VII/H],ul[VII/H],e_[VII/H],[VII/Fe],e_[VII/Fe],epscr,[Cr/H],ul[Cr/H],e_[Cr/H],[Cr/Fe],e_[Cr/Fe],epscrII,[CrII/H],ul[CrII/H],e_[CrII/H],[CrII/Fe],e_[CrII/Fe],epsmn,[Mn/H],ul[Mn/H],e_[Mn/H],[Mn/Fe],e_[Mn/Fe],epsfe,[Fe/H],ul[Fe/H],e_[Fe/H],[Fe/Fe],e_[Fe/Fe],epsfeII,[FeII/H],ul[FeII/H],e_[FeII/H],[FeII/Fe],e_[FeII/Fe],epsco,[Co/H],ul[Co/H],e_[Co/H],[Co/Fe],e_[Co/Fe],epsni,[Ni/H],ul[Ni/H],e_[Ni/H],[Ni/Fe],e_[Ni/Fe],epssr,[Sr/H],ul[Sr/H],e_[Sr/H],[Sr/Fe],e_[Sr/Fe],epsy,[Y/H],ul[Y/H],e_[Y/H],[Y/Fe],e_[Y/Fe],epszr,[Zr/H],ul[Zr/H],e_[Zr/H],[Zr/Fe],e_[Zr/Fe],epsba,[Ba/H],ul[Ba/H],e_[Ba/H],[Ba/Fe],e_[Ba/Fe],epsla,[La/H],ul[La/H],e_[La/H],[La/Fe],e_[La/Fe],epseu,[Eu/H],ul[Eu/H],e_[Eu/H],[Eu/Fe],e_[Eu/Fe],epsc,[C/H],ul[C/H],e_[C/H],[C/Fe],e_[C/Fe]
0,LMC-003,DW,3.36,-2.88,NaN,0.37,0.09,0.28,5.01,-2.59,NaN,0.26,0.38,0.21,2.51,-3.94,NaN,0.62,-0.97,0.65,4.19,-3.32,NaN,0.65,-0.35,0.62,2.51,-2.52,NaN,0.25,0.45,0.22,3.56,-2.78,NaN,0.24,0.19,0.22,0.02,-3.13,NaN,0.24,-0.16,0.29,2.15,-2.80,NaN,0.30,0.17,0.20,2.12,-2.83,NaN,0.26,0.14,0.32,1.03,NaN,-2.90,NaN,0.07,NaN,1.41,NaN,-2.52,NaN,0.45,NaN,2.26,-3.38,NaN,0.29,-0.41,0.20,2.73,-2.91,NaN,0.22,0.06,0.36,2.35,NaN,-3.08,NaN,-0.11,NaN,4.53,-2.97,NaN,0.30,0.0,0.20,4.49,-3.01,NaN,0.22,-0.04,0.34,2.06,-2.93,NaN,0.35,0.04,0.26,3.53,-2.69,NaN,0.27,0.28,0.21,-0.23,-3.10,NaN,0.29,-0.13,0.32,-0.67,NaN,-2.88,NaN,0.09,NaN,0.06,NaN,-2.52,NaN,0.45,NaN,-1.96,-4.14,NaN,0.24,-1.17,0.28,-1.73,NaN,-2.83,NaN,0.14,NaN,-2.21,NaN,-2.73,NaN,0.24,NaN,5.08,-3.35,NaN,0.40,-0.38,0.24
1,LMC-100,DW,3.49,-2.75,NaN,0.34,-0.08,0.23,5.32,-2.28,NaN,0.28,0.39,0.23,2.82,-3.63,NaN,0.67,-0.96,0.66,4.49,-3.02,NaN,0.85,-0.35,0.74,NaN,NaN,NaN,NaN,NaN,NaN,3.92,-2.42,NaN,0.27,0.25,0.24,0.61,-2.54,NaN,0.25,0.13,0.29,2.27,-2.68,NaN,0.32,-0.01,0.22,2.59,-2.36,NaN,0.25,0.31,0.30,1.10,-2.83,NaN,0.47,-0.16,0.38,1.86,NaN,-2.07,NaN,0.60,NaN,2.88,-2.76,NaN,0.31,-0.09,0.22,3.25,-2.39,NaN,0.24,0.28,0.36,2.59,-2.84,NaN,0.28,-0.17,0.23,4.83,-2.67,NaN,0.32,0.0,0.22,4.81,-2.69,NaN,0.25,-0.02,0.34,2.38,-2.61,NaN,0.36,0.06,0.23,3.56,-2.66,NaN,0.32,0.01,0.25,-0.82,-3.69,NaN,0.34,-1.02,0.35,-0.52,NaN,-2.73,NaN,-0.06,NaN,0.69,NaN,-1.89,NaN,0.78,NaN,-1.24,-3.42,NaN,0.27,-0.75,0.28,-1.27,NaN,-2.37,NaN,0.30,NaN,-1.78,NaN,-2.30,NaN,0.37,NaN,5.48,-2.95,NaN,0.40,-0.28,0.26
2,LMC-109,DW,3.21,-3.03,NaN,0.34,-0.18,0.21,5.18,-2.42,NaN,0.26,0.43,0.22,2.40,-4.05,NaN,0.66,-1.20,0.61,5.09,-2.42,NaN,0.69,0.43,0.61,2.79,-2.24,NaN,0.26,0.61,0.22,3.72,-2.62,NaN,0.26,0.23,0.22,0.39,-2.76,NaN,0.24,0.09,0.28,2.20,-2.75,NaN,0.32,0.10,0.20,2.46,-2.49,NaN,0.23,0.36,0.30,1.04,-2.89,NaN,0.31,-0.04,0.21,1.72,NaN,-2.21,NaN,0.64,NaN,2.52,-3.12,NaN,0.30,-0.27,0.20,2.93,-2.71,NaN,0.22,0.14,0.37,2.03,-3.40,NaN,0.28,-0.55,0.21,4.65,-2.85,NaN,0.32,0.0,0.20,4.66,-2.84,NaN,0.23,0.01,0.34,1.88,-3.11,NaN,0.34,-0.26,0.20,3.36,-2.86,NaN,0.27,-0.01,0.21,-0.22,-3.09,NaN,0.51,-0.24,0.49,-0.79,-3.00,NaN,0.24,-0.15,0.3,0.36,NaN,-2.22,NaN,0.63,NaN,-0.92,-3.10,NaN,0.26,-0.25,0.27,-1.35,NaN,-2.45,NaN,0.40,NaN,-1.85,-2.37,NaN,0.26,0.48,0.28,5.03,-3.40,NaN,0.39,-0.55,0.23
3,LMC-124,DW,3.39,-2.85,NaN,0.34,0.12,0.22,5.12,-2.48,NaN,0.26,0.49,0.23,3.00,-3.45,NaN,0.66,-0.48,0.62,4.79,-2.72,NaN,0.74,0.25,0.63,2.86,-2.17,NaN,0.26,0.80,0.22,3.74,-2.60,NaN,0.25,0.37,0.23,0.28,-2.87,NaN,0.24,0.10,0.30,2.22,-2.73,NaN,0.31,0.24,0.21,2.36,-2.59,NaN,0.25,0.38,0.31,1.12,NaN,-2.81,NaN,0.16,NaN,1.68,NaN,-2.26,NaN,0.71,NaN,2.44,-3.20,NaN,0.30,-0.23,0.22,NaN,NaN,NaN,NaN,NaN,NaN,2.38,NaN,-3.05,NaN,-0.08,NaN,4.53,-2.97,NaN,0.33,0.0,0.2

In [7]:
## Merge with the main table, made manually by combining Extended Data Table 1 and Extended Data Table 2
main_table = pd.read_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2024/chiti2024_Extended_Data_Table1_Table2.csv")

# Merge the two DataFrames on the 'Name' column
merged_df = pd.merge(main_table, df, on='Name', how='left')


## Print each colmumn that ends with a "_x" or "_y"
for col_xy in merged_df.columns:
    # col_xy = "e_[C/Fe]_x"
    if col_xy.endswith("_x"):
        col = col_xy[:-2]
        col_df = pd.DataFrame(columns=[col])
        col_df[col] = merged_df[col_xy]
        for i in range(len(merged_df[col_xy])):
            if i != merged_df[col+"_y"][i]:
                col_df[col] = merged_df[col + "_y"].combine_first(merged_df[col + "_x"])
                # print(col, i, merged_df[col+"_x"][i], merged_df[col+"_y"][i])

        merged_df[col] = col_df[col]
        merged_df = merged_df.drop(columns=[col+'_x', col+'_y'])

## Set 'Loc' to 'DW' for all rows
merged_df['Loc'] = 'DW'

## Set 'Ref' and 'Reference' columns
merged_df['Ref'] = 'CHI24'
merged_df['Reference'] = 'Chiti+2024'

## Add the 'RA_deg' and 'DEC_deg' columns
merged_df['RA_deg'] = merged_df['RA_hms'].apply(coord.ra_hms_to_deg)
merged_df['DEC_deg'] = merged_df['DEC_dms'].apply(coord.dec_dms_to_deg)
merged_df['RA_deg'] = merged_df['RA_deg'].apply(lambda x: normal_round(float(x), 4))
merged_df['DEC_deg'] = merged_df['DEC_deg'].apply(lambda x: normal_round(float(x), 4))

## Classify the stars 'Ncap_key' and 'C_key'
merged_df['MP_key'] = np.nan
merged_df['alpha_key'] = np.nan
merged_df['Ncap_key'] = np.nan
merged_df['C_key'] = np.nan

for name in merged_df['Name']:
    FeH =  merged_df[merged_df['Name'] == name]['[Fe/H]'].values[0] if '[Fe/H]' in merged_df.columns else np.nan
    CFe =  merged_df[merged_df['Name'] == name]['[C/Fe]'].values[0] if '[C/Fe]' in merged_df.columns else np.nan
    MgFe = merged_df[merged_df['Name'] == name]['[Mg/Fe]'].values[0] if '[Mg/Fe]' in merged_df.columns else np.nan
    SiFe = merged_df[merged_df['Name'] == name]['[Si/Fe]'].values[0] if '[Si/Fe]' in merged_df.columns else np.nan
    CaFe = merged_df[merged_df['Name'] == name]['[Ca/Fe]'].values[0] if '[Ca/Fe]' in merged_df.columns else np.nan
    TiFe = merged_df[merged_df['Name'] == name]['[Ti/Fe]'].values[0] if '[Ti/Fe]' in merged_df.columns else np.nan
    EuFe = merged_df[merged_df['Name'] == name]['[Eu/Fe]'].values[0] if '[Eu/Fe]' in merged_df.columns else np.nan
    BaFe = merged_df[merged_df['Name'] == name]['[Ba/Fe]'].values[0] if '[Ba/Fe]' in merged_df.columns else np.nan
    SrFe = merged_df[merged_df['Name'] == name]['[Sr/Fe]'].values[0] if '[Sr/Fe]' in merged_df.columns else np.nan
    PbFe = merged_df[merged_df['Name'] == name]['[Pb/Fe]'].values[0] if '[Pb/Fe]' in merged_df.columns else np.nan
    LaFe = merged_df[merged_df['Name'] == name]['[La/Fe]'].values[0] if '[La/Fe]' in merged_df.columns else np.nan
    HfFe = merged_df[merged_df['Name'] == name]['[Hf/Fe]'].values[0] if '[Hf/Fe]' in merged_df.columns else np.nan
    IrFe = merged_df[merged_df['Name'] == name]['[Ir/Fe]'].values[0] if '[Ir/Fe]' in merged_df.columns else np.nan

    ## Classify the stars and fill in the Sci_key columns if they are not already filled
    mp_val = merged_df.loc[merged_df['Name'] == name, 'MP_key'].values[0]
    if pd.isna(mp_val) or mp_val == '' or (isinstance(mp_val, float) and np.isnan(mp_val)):
        merged_df.loc[merged_df['Name'] == name, 'MP_key'] = classify_metallicity(FeH)
  
    alpha_val = merged_df.loc[merged_df['Name'] == name, 'alpha_key'].values[0]
    if pd.isna(alpha_val) or alpha_val == '' or (isinstance(alpha_val, float) and np.isnan(alpha_val)):
        if not any(pd.isna(val) for val in [MgFe, SiFe, CaFe, TiFe]):
            merged_df.loc[merged_df['Name'] == name, 'alpha_key'] = classify_alpha_enhancement(MgFe, SiFe, CaFe, TiFe)

    ncap_val = merged_df.loc[merged_df['Name'] == name, 'Ncap_key'].values[0]
    if pd.isna(ncap_val) or ncap_val == '' or (isinstance(ncap_val, float) and np.isnan(ncap_val)):
        merged_df.loc[merged_df['Name'] == name, 'Ncap_key'] = classify_neutron_capture(EuFe, BaFe, SrFe, PbFe, LaFe, HfFe, IrFe)

    c_val = merged_df.loc[merged_df['Name'] == name, 'C_key'].values[0]
    if pd.isna(c_val) or c_val == '' or (isinstance(c_val, float) and np.isnan(c_val)):
        merged_df.loc[merged_df['Name'] == name, 'C_key'] = classify_carbon_enhancement(CFe, BaFe)

## Replace 'ul_[{}/H]' with 'ul[{}/H]' and 'ul_[{}/Fe]' with 'ul[{}/Fe]'
for col in merged_df.columns:
    if col.startswith('ul_['):
        new_col = col.replace('ul_[', 'ul[')
        merged_df.rename(columns={col: new_col}, inplace=True)

## Moving ulimit values to the ul[{}/Fe] columns, from the [{}/Fe] columns
for col in merged_df.columns:
    if col.startswith('ul[') and col.endswith('/Fe]'):
        elem = col.split('/')[0][3:]

        mask0 = merged_df[col] == 0
        if mask0.any():
            merged_df.loc[mask0, f'ul[{elem}/Fe]'] = np.nan

        mask1 = merged_df[col] == 1
        if mask1.any():
            merged_df.loc[mask1, f'ul[{elem}/Fe]'] = merged_df.loc[mask1, f'[{elem}/Fe]'].values
            merged_df.loc[mask1, f'[{elem}/Fe]'] = np.nan



## Sort the DataFrame by 'RA_deg' and 'DEC_deg'
merged_df = merged_df.sort_values(by=['RA_deg', 'DEC_deg'])

## Re-order the columns
meta_cols = [
    'Name','Ref','Reference','Loc','MP_key','alpha_key','Ncap_key','C_key',
    'RA_hms','DEC_dms','RA_deg','DEC_deg','RV','e_RV',
    'Teff','e_Teff','logg','e_logg','vmic','e_vmic',
    'G_mag','t_exp','Date','Instrument'
]

abund_cols = [
    'epsc'   , '[C/H]'   , 'ul[C/H]'   , 'e_[C/H]'   , '[C/Fe]'   , 'ul[C/Fe]'  , 'e_[C/Fe]' , '[C/Fe]_c',    # Z=6
    'epsna'  , '[Na/H]'  , 'ul[Na/H]'  , 'e_[Na/H]'  , '[Na/Fe]'  , 'e_[Na/Fe]'  ,                             # Z=11
    'epsmg'  , '[Mg/H]'  , 'ul[Mg/H]'  , 'e_[Mg/H]'  , '[Mg/Fe]'  , 'e_[Mg/Fe]'  ,                             # Z=12
    'epsal'  , '[Al/H]'  , 'ul[Al/H]'  , 'e_[Al/H]'  , '[Al/Fe]'  , 'e_[Al/Fe]'  ,                             # Z=13
    'epssi'  , '[Si/H]'  , 'ul[Si/H]'  , 'e_[Si/H]'  , '[Si/Fe]'  , 'e_[Si/Fe]'  ,                             # Z=14
    'epsk'   , '[K/H]'   , 'ul[K/H]'   , 'e_[K/H]'   , '[K/Fe]'   , 'e_[K/Fe]'   ,                             # Z=19
    'epsca'  , '[Ca/H]'  , 'ul[Ca/H]'  , 'e_[Ca/H]'  , '[Ca/Fe]'  , 'e_[Ca/Fe]'  ,                             # Z=20
    'epssc'  , '[Sc/H]'  , 'ul[Sc/H]'  , 'e_[Sc/H]'  , '[Sc/Fe]'  , 'e_[Sc/Fe]'  ,                             # Z=21
    'epsti'  , '[Ti/H]'  , 'ul[Ti/H]'  , 'e_[Ti/H]'  , '[Ti/Fe]'  , 'e_[Ti/Fe]'  ,                             # Z=22
    'epstiII', '[TiII/H]', 'ul[TiII/H]', 'e_[TiII/H]', '[TiII/Fe]', 'e_[TiII/Fe]',                             # Z=22 (ionized)
    'epsv'   , '[V/H]'   , 'ul[V/H]'   , 'e_[V/H]'   , '[V/Fe]'   , 'e_[V/Fe]'   ,                             # Z=23
    'epsvII' , '[VII/H]' , 'ul[VII/H]' , 'e_[VII/H]' , '[VII/Fe]' , 'e_[VII/Fe]' ,                             # Z=23 (ionized)
    'epscr'  , '[Cr/H]'  , 'ul[Cr/H]'  , 'e_[Cr/H]'  , '[Cr/Fe]'  , 'e_[Cr/Fe]'  ,                             # Z=24
    'epscrII', '[CrII/H]', 'ul[CrII/H]', 'e_[CrII/H]', '[CrII/Fe]', 'e_[CrII/Fe]',                             # Z=24 (ionized)
    'epsmn'  , '[Mn/H]'  , 'ul[Mn/H]'  , 'e_[Mn/H]'  , '[Mn/Fe]'  , 'e_[Mn/Fe]'  ,                             # Z=25
    'epsfe'  , '[Fe/H]'  , 'ul[Fe/H]'  , 'e_[Fe/H]'  , '[Fe/Fe]'  , 'e_[Fe/Fe]'  ,                             # Z=26
    'epsfeII', '[FeII/H]', 'ul[FeII/H]', 'e_[FeII/H]', '[FeII/Fe]', 'e_[FeII/Fe]',                             # Z=26 (ionized)
    'epsco'  , '[Co/H]'  , 'ul[Co/H]'  , 'e_[Co/H]'  , '[Co/Fe]'  , 'e_[Co/Fe]'  ,                             # Z=27
    'epsni'  , '[Ni/H]'  , 'ul[Ni/H]'  , 'e_[Ni/H]'  , '[Ni/Fe]'  , 'e_[Ni/Fe]'  ,                             # Z=28
    'epssr'  , '[Sr/H]'  , 'ul[Sr/H]'  , 'e_[Sr/H]'  , '[Sr/Fe]'  , 'e_[Sr/Fe]'  ,                             # Z=38
    'epsy'   , '[Y/H]'   , 'ul[Y/H]'   , 'e_[Y/H]'   , '[Y/Fe]'   , 'e_[Y/Fe]'   ,                             # Z=39
    'epszr'  , '[Zr/H]'  , 'ul[Zr/H]'  , 'e_[Zr/H]'  , '[Zr/Fe]'  , 'e_[Zr/Fe]'  ,                             # Z=40
    'epsba'  , '[Ba/H]'  , 'ul[Ba/H]'  , 'e_[Ba/H]'  , '[Ba/Fe]'  , 'e_[Ba/Fe]'  ,                             # Z=56
    'epsla'  , '[La/H]'  , 'ul[La/H]'  , 'e_[La/H]'  , '[La/Fe]'  , 'e_[La/Fe]'  ,                             # Z=57
    'epseu'  , '[Eu/H]'  , 'ul[Eu/H]'  , 'e_[Eu/H]'  , '[Eu/Fe]'  , 'ul[Eu/Fe]' , 'e_[Eu/Fe]'                 # Z=63
]

merged_df2 = merged_df[meta_cols + abund_cols]
display(merged_df2)
# display(merged_df2[['Name', '[Fe/H]', '[C/Fe]', 'ul[C/Fe]', 'Ncap_key', 'Reference']].sort_values(by='[Fe/H]'))

,Name,Ref,Reference,Loc,MP_key,alpha_key,Ncap_key,C_key,RA_hms,DEC_dms,RA_deg,DEC_deg,RV,e_RV,Teff,e_Teff,logg,e_logg,vmic,e_vmic,G_mag,t_exp,Date,Instrument,epsc,[C/H],ul[C/H],e_[C/H],[C/Fe],ul[C/Fe],e_[C/Fe],[C/Fe]_c,epsna,[Na/H],ul[Na/H],e_[Na/H],[Na/Fe],e_[Na/Fe],epsmg,[Mg/H],ul[Mg/H],e_[Mg/H],[Mg/Fe],e_[Mg/Fe],epsal,[Al/H],ul[Al/H],e_[Al/H],[Al/Fe],e_[Al/Fe],epssi,[Si/H],ul[Si/H],e_[Si/H],[Si/Fe],e_[Si/Fe],epsk,[K/H],ul[K/H],e_[K/H],[K/Fe],e_[K/Fe],epsca,[Ca/H],ul[Ca/H],e_[Ca/H],[Ca/Fe],e_[Ca/Fe],epssc,[Sc/H],ul[Sc/H],e_[Sc/H],[Sc/Fe],e_[Sc/Fe],epsti,[Ti/H],ul[Ti/H],e_[Ti/H],[Ti/Fe],e_[Ti/Fe],epstiII,[TiII/H],ul[TiII/H],e_[TiII/H],[TiII/Fe],e_[TiII/Fe],epsv,[V/H],ul[V/H],e_[V/H],[V/Fe],e_[V/Fe],epsvII,[VII/H],ul[VII/H],e_[VII/H],[VII/Fe],e_[VII/Fe],epscr,[Cr/H],ul[Cr/H],e_[Cr/H],[Cr/Fe],e_[Cr/Fe],epscrII,[CrII/H],ul[CrII/H],e_[CrII/H],[CrII/Fe],e_[CrII/Fe],epsmn,[Mn/H],ul[Mn/H],e_[Mn/H],[Mn/Fe],e_[Mn/Fe],epsfe,[Fe/H],ul[Fe/H],e_[Fe/H],[Fe/Fe],e_[Fe/Fe],epsfeII,[FeII/H],ul[FeII/H],e_[FeII/H],[FeII/Fe],e_[FeII/Fe],epsco,[Co/H],ul[Co/H],e_[Co/H],[Co/Fe],e_[Co/Fe],epsni,[Ni/H],ul[Ni/H],e_[Ni/H],[Ni/Fe],e_[Ni/Fe],epssr,[Sr/H],ul[Sr/H],e_[Sr/H],[Sr/Fe],e_[Sr/Fe],epsy,[Y/H],ul[Y/H],e_[Y/H],[Y/Fe],e_[Y/Fe],epszr,[Zr/H],ul[Zr/H],e_[Zr/H],[Zr/Fe],e_[Zr/Fe],epsba,[Ba/H],ul[Ba/H],e_[Ba/H],[Ba/Fe],e_[Ba/Fe],epsla,[La/H],ul[La/H],e_[La/H],[La/Fe],e_[La/Fe],epseu,[Eu/H],ul[Eu/H],e_[Eu/H],[Eu/Fe],ul[Eu/Fe],e_[Eu/Fe]
6,LMC-109,CHI24,Chiti+2024,DW,VMP,,R1,,04:12:01.964,-66:00:58.36,63.0082,-66.0162,222.5,2.0,4504.0,155.0,1.10,0.31,2.58,0.28,17.01,20; 40,2022-12-19; 2022-12-20,MIKE,5.03,-3.40,NaN,0.39,-0.55,NaN,0.23,0.19,3.21,-3.03,NaN,0.34,-0.18,0.21,5.18,-2.42,NaN,0.26,0.43,0.22,2.40,-4.05,NaN,0.66,-1.20,0.61,5.09,-2.42,NaN,0.69,0.43,0.61,2.79,-2.24,NaN,0.26,0.61,0.22,3.72,-2.62,NaN,0.26,0.23,0.22,0.39,-2.76,NaN,0.24,0.09,0.28,2.20,-2.75,NaN,0.32,0.10,0.20,2.46,-2.49,NaN,0.23,0.36,0.30,1.04,-2.89,NaN,0.31,-0.04,0.21,1.72,NaN,-2.21,NaN,0.64,NaN,2.52,-3.12,NaN,0.30,-0.27,0.20,2.93,-2.71,NaN,0.22,0.14,0.37,2.03,-3.40,NaN,0.28,-0.55,0.21,4.65,-2.85,NaN,0.32,0.0,0.20,4.66,-2.84,NaN,0.23,0.01,0.34,1.88,-3.11,NaN,0.34,-0.26,0.20,3.36,-2.86,NaN,0.27,-0.01,0.21,-0.22,-3.09,NaN,0.51,-0.24,0.49,-0.79,-3.00,NaN,0.24,-0.15,0.3,0.36,NaN,-2.22,NaN,0.63,NaN,-0.92,-3.10,NaN,0.26,-0.25,0.27,-1.35,NaN,-2.45,NaN,0.40,NaN,-1.85,-2.37,NaN,0.26,0.48,NaN,0.28
19,LMC-402,CHI24,Chiti+2024,DW,EMP,NaN,,,04:28:27.512,-69:22:08.36,67.1146,-69.3690,NaN,NaN,4545.0,150.0,1.12,0.30,2.14,0.20,17.21,20,2023-01-27,MIKE,NaN,NaN,NaN,NaN,NaN,-0.10,NaN,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.20,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,LMC-204,CHI24,Chiti+2024,DW,VMP,,R1,,04:33:20.143,-68:02:41.87,68.3339,-68.0450,252.3,2.0,4594.0,173.0,0.95,0.36,3.02,0.34,17.30,47,2022-12-01,MIKE,5.45,-2.98,NaN,0.46,-0.15,NaN,0.31,0.59,3.16,-3.08,NaN,0.35,-0.25,0.27,5.08,-2.52,NaN,0.31,0.31,0.28,2.03,-4.42,NaN,0.64,-1.59,0.63,4.81,-2.70,NaN,0.78,0.13,0.73,2.76,-2.27,NaN,0.31,0.56,0.28,3.60,-2.74,NaN,0.30,0.09,0.29,0.17,-2.98,NaN,0.30,-0.15,0.33,2.26,-2.69,NaN,0.35,0.14,0.27,2.21,-2.74,NaN,0.30,0.09,0.34,1.36,NaN,-2.57,NaN,0.26,NaN,1.69,NaN,-2.24,NaN,0.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.72,NaN,-2.71,NaN,0.12,NaN,4.67,-2.83,NaN,0.35,0.0,0.27,4.61,-2.89,NaN,0.29,-0.06,0.38,1.91,-3.08,NaN,0.35,-0.25,0.27,3.14,-3.08,NaN,0.35,-0.25,0.27,-1.40,-4.27,NaN,0.30,-1.44,0.34,-0.57,NaN,-2.78,NaN,0.05,NaN,0.40,NaN,-2.18,NaN,0.65,NaN,-1.67,-3.85,NaN,0.31,-1.02,0.32,-1.12,NaN,-2.22,NaN,0.61,Na

In [8]:
merged_df2.to_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2024/chiti2024_yelland.csv", index=False)